# Finding NetCDF-4 collections

This notebook shows how to use earthaccess to discover NASA Earthdata collections that provide granules in netCDF-4 format. In the next step, it opens a representative netCDF-4 file from each collection to inspect and list the available variable names.

In [10]:
import earthaccess
import pandas as pd
import numpy as np

from typing import Dict, Optional, Tuple, Any

# ----------------------------------------
# Helpers to parse metadata from earthaccess
# ----------------------------------------


def _parse_temporal(umm: Dict[str, Any]) -> Tuple[Optional[str], Optional[str]]:
    temporal = umm.get("TemporalExtents", [])
    rng = (temporal or [{}])[0].get("RangeDateTimes", [])
    begin = (rng or [{}])[0].get("BeginningDateTime")
    end = (rng or [{}])[0].get("EndingDateTime")
    return begin, end


def _parse_bounds_from_spatial(
    umm: Dict[str, Any],
) -> Tuple[Optional[float], Optional[float], Optional[float], Optional[float]]:
    spatial = umm.get("SpatialExtent", {}) or {}
    horiz = spatial.get("HorizontalSpatialDomain", {}) or {}
    geom = horiz.get("Geometry", {}) or {}

    # 1) Bounding rectangles
    rects = geom.get("BoundingRectangles") or []
    if rects:
        wests = [r.get("WestBoundingCoordinate") for r in rects if r]
        easts = [r.get("EastBoundingCoordinate") for r in rects if r]
        souths = [r.get("SouthBoundingCoordinate") for r in rects if r]
        norths = [r.get("NorthBoundingCoordinate") for r in rects if r]
        if all(len(lst) > 0 for lst in (wests, easts, souths, norths)):
            return (
                float(np.min(wests)),
                float(np.min(souths)),
                float(np.max(easts)),
                float(np.max(norths)),
            )

    # 2) GPolygons
    gpolys = geom.get("GPolygons") or []
    coords_w, coords_e, coords_s, coords_n = [], [], [], []
    for gp in gpolys:
        b = gp.get("Boundary", {})
        pts = b.get("Points", [])
        lons = [p.get("Longitude") for p in pts if p and p.get("Longitude") is not None]
        lats = [p.get("Latitude") for p in pts if p and p.get("Latitude") is not None]
        if lons and lats:
            coords_w.append(np.min(lons))
            coords_e.append(np.max(lons))
            coords_s.append(np.min(lats))
            coords_n.append(np.max(lats))
    if coords_w and coords_e and coords_s and coords_n:
        return (
            float(np.min(coords_w)),
            float(np.min(coords_s)),
            float(np.max(coords_e)),
            float(np.max(coords_n)),
        )

    return None, None, None, None

First, let's find all collections that provide netCDF-4 files using the `earthaccess` library.

In [11]:
%%time

# step 1-a: search collections with netcdf-4

query = earthaccess.DataCollections()
query.params["granule_data_format"] = "*netcdf-4*"
query.option("granule_data_format", "pattern", True)
results = query.get_all()
print(f"Number of collections found: {len(results)}")

Number of collections found: 1934
CPU times: user 371 ms, sys: 41.7 ms, total: 413 ms
Wall time: 4.54 s


Next, parse metadata for each collection to find a temporal and spatial range. 

In [12]:
%%time

# step 1-b: parse metadata to find temporal and spatial bounds and save to csv
rows = []
for rec in results:
    meta = rec.get("meta", {}) or {}
    umm = rec.get("umm", {}) or {}
    concept_id = meta.get("concept-id") or meta.get("concept_id")
    short_name = umm.get("ShortName")
    entry_title = umm.get("EntryTitle")
    provider_id = meta.get("provider-id")

    begin, end = _parse_temporal(umm)
    west, south, east, north = _parse_bounds_from_spatial(umm)

    rows.append(
        {
            "concept_id": concept_id,
            "short_name": short_name,
            "entry_title": entry_title,
            "provider_id": provider_id,
            "begin_time": begin,
            "end_time": end,
            "west": west,
            "south": south,
            "east": east,
            "north": north,
        }
    )

df = pd.DataFrame(rows)

print(df.head())

concept_ids = [r["concept_id"] for r in rows if r["concept_id"]]

out_csv = "output/cmr_collections_netcdf4.csv"
df.to_csv(out_csv, index=False)

            concept_id            short_name  \
0    C2105092163-LAADS              VNP03IMG   
1  C2545314550-LPCLOUD                 VNP21   
2    C2105091501-LAADS              VNP02IMG   
3    C1593392869-LAADS  CLDMSK_L2_MODIS_Aqua   
4    C1562021084-LAADS  CLDMSK_L2_VIIRS_SNPP   

                                         entry_title provider_id  \
0  VIIRS/NPP Imagery Resolution Terrain Corrected...       LAADS   
1  VIIRS/NPP Land Surface Temperature and Emissiv...     LPCLOUD   
2  VIIRS/NPP Imagery Resolution 6-Min L1B Swath 3...       LAADS   
3           MODIS/Aqua Cloud Mask 5-Min Swath 1000 m       LAADS   
4       VIIRS/Suomi-NPP Cloud Mask 6-Min Swath 750 m       LAADS   

                 begin_time end_time   west  south   east  north  
0  2012-01-19T00:00:00.000Z     None -180.0  -90.0  180.0   90.0  
1  2012-01-17T00:00:00.000Z     None -180.0  -90.0  180.0   90.0  
2  2012-01-19T00:00:00.000Z     None -180.0  -90.0  180.0   90.0  
3  2002-07-04T00:00:00.000Z     No

Next, open a representative netcdf-4 file from each collection and list variable names


In [13]:
import concurrent.futures
import earthaccess
from urllib.parse import urlparse
import pandas as pd
import xarray as xr
from datetime import datetime, timezone


df = pd.read_csv("output/cmr_collections_netcdf4.csv")

for col in ["links", "variables", "status", "error", "scheme"]:
    df[col] = None


def _pick_best_link(all_links):
    """Prefer HTTPS; else S3; else None."""
    https = [u for u in all_links if u.startswith("http")]
    s3 = [u for u in all_links if u.startswith("s3://")]
    if s3:
        return s3[0]
    if https:
        return https[0]
    return None


def _open_xarray_dataset(url):
    """Open a NetCDF URL that may be HTTPS or S3 and return (ds, scheme)."""
    scheme = urlparse(url).scheme.lower()
    if scheme in ("http", "https"):
        fs = earthaccess.get_fsspec_https_session()
        return xr.open_dataset(
            fs.open(url), engine="h5netcdf", decode_times=False
        ), "https"
    elif scheme == "s3":
        s3 = earthaccess.get_s3fs_session()
        return xr.open_dataset(
            s3.open(url, "rb"), engine="h5netcdf", decode_times=False
        ), "s3"
    else:
        raise ValueError(f"Unsupported URL scheme: {scheme}")


def process_row(i_row):
    i, row = i_row
    concept_id = row["concept_id"]
    begin = row["begin_time"]
    end = (
        row["end_time"]
        if pd.notna(row["end_time"])
        else datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    )

    logs = []
    logs.append(f"\n🔍 [{i}] Concept ID: {concept_id}")
    logs.append(f"   🚀 [{i}] Starting search for {concept_id}...")

    try:
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as ex:
            fut = ex.submit(
                earthaccess.search_data,
                concept_id=concept_id,
                temporal=(begin, end),
                count=1,
            )
            results = fut.result(timeout=120)
    except concurrent.futures.TimeoutError:
        logs.append(f"   ⏳ [{i}] Timeout while searching {concept_id}")
        return {
            "i": i,
            "concept_id": concept_id,
            "links": None,
            "variables": None,
            "status": "timeout",
            "error": None,
            "scheme": None,
            "logs": logs,
        }
    except Exception as e:
        logs.append(f"   ❌ [{i}] Search failed for {concept_id}: {e}")
        return {
            "i": i,
            "concept_id": concept_id,
            "links": None,
            "variables": None,
            "status": "search_failed",
            "error": str(e),
            "scheme": None,
            "logs": logs,
        }

    if not results:
        logs.append(f"   ⚠️  [{i}] No granules for {concept_id}")
        return {
            "i": i,
            "concept_id": concept_id,
            "links": None,
            "variables": None,
            "status": "no_granules",
            "error": None,
            "scheme": None,
            "logs": logs,
        }

    try:
        all_links = results[0].data_links() or []
    except Exception as e:
        logs.append(f"   ⚠️  [{i}] Could not extract data_links: {e}")
        return {
            "i": i,
            "concept_id": concept_id,
            "links": None,
            "variables": None,
            "status": "no_links",
            "error": str(e),
            "scheme": None,
            "logs": logs,
        }

    netcdf_url = _pick_best_link(all_links)
    if not netcdf_url:
        logs.append(f"   ⚠️  [{i}] No usable HTTPS/S3 NetCDF links for {concept_id}")
        return {
            "i": i,
            "concept_id": concept_id,
            "links": None,
            "variables": None,
            "status": "no_links",
            "error": None,
            "scheme": None,
            "logs": logs,
        }

    logs.append(f"   🔗 [{i}] Link chosen: {netcdf_url}")

    try:
        ds, scheme = _open_xarray_dataset(netcdf_url)
        with ds:
            variables = list(ds.data_vars.keys())
        logs.append(f"   📊 [{i}] Variables ({len(variables)}): {variables}")
        logs.append(f"   ✅ [{i}] Result: ok, scheme: {scheme}")
        return {
            "i": i,
            "concept_id": concept_id,
            "links": netcdf_url,
            "variables": variables,
            "status": "ok",
            "error": None,
            "scheme": scheme,
            "logs": logs,
        }
    except Exception as e:
        logs.append(f"   ⚠️  [{i}] Failed to open dataset: {e}")
        return {
            "i": i,
            "concept_id": concept_id,
            "links": netcdf_url,
            "variables": [],
            "status": "open_failed",
            "error": str(e),
            "scheme": urlparse(netcdf_url).scheme or None,
            "logs": logs,
        }


# ----------------------------
# Run in parallel
# ----------------------------
rows = []
n = max(10, len(df))
print(f"\n🚀 Starting processing of {n} rows...", flush=True)
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(process_row, item) for item in df.iloc[:n].iterrows()]
    for fut in concurrent.futures.as_completed(futures):
        res = fut.result()

        # Print all logs for this collection at once
        for log in res.get("logs", []):
            print(log, flush=True)

        rows.append({k: v for k, v in res.items() if k != "logs"})

out = pd.DataFrame(rows).set_index("i").sort_index()

# Merge back into original df (preserves all other columns)
df.loc[out.index, ["links", "variables", "status", "error", "scheme"]] = out[
    ["links", "variables", "status", "error", "scheme"]
]

print("\n📦 Merge complete. Sample:", flush=True)
print(df.loc[out.index, ["concept_id", "scheme", "links", "status"]].head(), flush=True)


🚀 Starting processing of 1934 rows...

🔍 [2] Concept ID: C2105091501-LAADS
   🚀 [2] Starting search for C2105091501-LAADS...
   🔗 [2] Link chosen: https://data.laadsdaac.earthdatacloud.nasa.gov/prod-lads/VNP02IMG/VNP02IMG.A2012019.0000.002.2020318151901.nc
   📊 [2] Variables (0): []
   ✅ [2] Result: ok, scheme: https

🔍 [1] Concept ID: C2545314550-LPCLOUD
   🚀 [1] Starting search for C2545314550-LPCLOUD...
   🔗 [1] Link chosen: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VNP21.002/VNP21.A2012019.0000.002.2023123113154/VNP21.A2012019.0000.002.2023123113154.nc
   📊 [1] Variables (0): []
   ✅ [1] Result: ok, scheme: https

🔍 [4] Concept ID: C1562021084-LAADS
   🚀 [4] Starting search for C1562021084-LAADS...
   🔗 [4] Link chosen: https://data.laadsdaac.earthdatacloud.nasa.gov/prod-lads/CLDMSK_L2_VIIRS_SNPP/CLDMSK_L2_VIIRS_SNPP.A2012061.0000.001.2019070194123.nc
   📊 [4] Variables (0): []
   ✅ [4] Result: ok, scheme: https

🔍 [5] Concept ID: C1964798938-LAADS
   🚀 [5] Sta

/tmp/ipykernel_429/1565676243.py:31: UserWarning: The 'phony_dims' kwarg now defaults to 'access'. Previously 'phony_dims=None' would raise an error. For full netcdf equivalence please use phony_dims='sort'.
  return xr.open_dataset(
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:204: SerializationWarning: variable 'lat' has multiple fill values {np.float32(1e+30), np.float64(1e+30)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:204: SerializationWarning: variable 'lon' has multiple fill values {np.float32(1e+30), np.float64(1e+30)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:204: SerializationWarning: variable 'model_speed' has multiple fill values {np.float32(1e+30), np.float64(1e+30)} defined, decoding all values to NaN.
  var = coder.decode(var, n


🔍 [30] Concept ID: C2545310869-LPCLOUD
   🚀 [30] Starting search for C2545310869-LPCLOUD...
   🔗 [30] Link chosen: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VJ114.002/VJ114.A2018005.0000.002.2022252033022/VJ114.A2018005.0000.002.2022252033022.nc
   📊 [30] Variables (35): ['CMG_night', 'FP_AdjCloud', 'FP_AdjWater', 'FP_CMG_col', 'FP_CMG_row', 'FP_MAD_DT', 'FP_MAD_R7', 'FP_MAD_T13', 'FP_MAD_T15', 'FP_MeanDT', 'FP_MeanR7', 'FP_MeanT13', 'FP_MeanT15', 'FP_NumValid', 'FP_R7', 'FP_RelAzAng', 'FP_SolZenAng', 'FP_T13', 'FP_T15', 'FP_ViewZenAng', 'FP_WinSize', 'FP_confidence', 'FP_land', 'FP_latitude', 'FP_line', 'FP_longitude', 'FP_power', 'FP_sample', 'algorithm QA', 'fire mask', 'qhist07', 'qhist11', 'qhist13', 'qhist15', 'qhist16']
   ✅ [30] Result: ok, scheme: https


/tmp/ipykernel_429/1565676243.py:31: UserWarning: The 'phony_dims' kwarg now defaults to 'access'. Previously 'phony_dims=None' would raise an error. For full netcdf equivalence please use phony_dims='sort'.
  return xr.open_dataset(



🔍 [35] Concept ID: C3392966952-OB_CLOUD
   🚀 [35] Starting search for C3392966952-OB_CLOUD...
   🔗 [35] Link chosen: https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240305T000858.L1B.V3.nc
   📊 [35] Variables (0): []
   ✅ [35] Result: ok, scheme: https

🔍 [39] Concept ID: C2859232902-LAADS
   🚀 [39] Starting search for C2859232902-LAADS...
   🔗 [39] Link chosen: https://data.laadsdaac.earthdatacloud.nasa.gov/prod-lads/XAERDT_L2_VIIRS_SNPP/XAERDT_L2_VIIRS_SNPP.A2019001.0012.001.2023150212003.nc
   📊 [39] Variables (0): []
   ✅ [39] Result: ok, scheme: https

🔍 [40] Concept ID: C3406446219-OB_CLOUD
   🚀 [40] Starting search for C3406446219-OB_CLOUD...
   🔗 [40] Link chosen: https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/S3A_OLCI_EFRNT.20160425T113330.L2.OC.nc
   📊 [40] Variables (0): []
   ✅ [40] Result: ok, scheme: https

🔍 [36] Concept ID: C2930761273-LARC_CLOUD
   🚀 [36] Starting search for C2930761273-LARC_CLOUD...
   🔗 [36] Link chose

...


🔍 [1557] Concept ID: C3215607563-LARC_CLOUD
   🚀 [1557] Starting search for C3215607563-LARC_CLOUD...
   🔗 [1557] Link chosen: https://data.asdc.earthdata.nasa.gov/asdc-prod-protected/TES/TL2CH4LN.7/2004.08.01/TES-Aura_L2-CH4-Nadir_2004-08_v007_Lite-v02.00.nc
   📊 [1557] Variables (31): ['Altitude', 'AveragingKernel', 'Column750', 'Column750_Averagingkernel', 'Column750_Constraintvector', 'Column750_Initial', 'Column750_ObservationError', 'Column750_PWF', 'ConstraintVector', 'DOFs', 'DayNightFlag', 'GlobalSurveyFlag', 'LandFlag', 'Latitude', 'Longitude', 'ObservationErrorCovariance', 'Pressure', 'Quality', 'Run', 'Scan', 'Sequence', 'RTVMR', 'RTVMR_Pressure', 'RTVMR_ErrorObservation', 'Species', 'SurfaceAltitude', 'Time', 'UT_Hour', 'YearFloat', 'YYYYMMDD', 'SoundingID']
   ✅ [1557] Result: ok, scheme: https

🔍 [1560] Concept ID: C3215610255-LARC_CLOUD
   🚀 [1560] Starting search for C3215610255-LARC_CLOUD...
   🔗 [1560] Link chosen: https://data.asdc.earthdata.nasa.gov/asdc-prod-prot

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/namedarray/core.py:261: UserWarning: Duplicate dimension names present: dimensions {'Grid_Pressure_Composite'} appear more than once in dims=('Grid_Targets', 'Grid_Pressure_Composite', 'Grid_Pressure_Composite'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recommend you rename the dims immediately to become distinct, as most xarray functionality is likely to fail silently if you do not. To rename the dimensions you will need to set the ``.dims`` attribute of each variable, ``e.g. var.dims=('x0', 'x1')``.
  self._dims = self._parse_dimensions(dims)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/namedarray/core.py:261: UserWarning: Duplicate dimension names present: dimensions {'Grid_Pressure'} appear more than once in dims=('Grid_Targets', 'Grid_Pressure', 'Grid_Pressure'). We do not yet support duplicate dimension names, but we do allow initial construc

   📊 [1562] Variables (25): ['Altitude', 'AveragingKernel', 'ConstraintVector', 'DOFs', 'DayNightFlag', 'GlobalSurveyFlag', 'LandFlag', 'Latitude', 'Longitude', 'ObservationErrorCovariance', 'Pressure', 'Quality', 'Run', 'Scan', 'Sequence', 'RTVMR', 'RTVMR_Pressure', 'RTVMR_ErrorObservation', 'Species', 'SurfaceAltitude', 'Time', 'UT_Hour', 'YearFloat', 'YYYYMMDD', 'SoundingID']
   ✅ [1562] Result: ok, scheme: https

🔍 [1568] Concept ID: C2342535056-LARC_ASDC
   🚀 [1568] Starting search for C2342535056-LARC_ASDC...
   🔗 [1568] Link chosen: https://asdc.larc.nasa.gov/data/TRACER-AQ/Ground_LaPorte_Data_1/TRACERAQ-GSFCCEILOMETER_LAPORTE_README_R0.pdf
   ⚠️  [1568] Failed to open dataset: b'%PDF-1.7' is not the signature of a valid netCDF4 file

🔍 [1564] Concept ID: C2036879048-POCLOUD
   🚀 [1564] Starting search for C2036879048-POCLOUD...
   🔗 [1564] Link chosen: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/TMI-REMSS-L2P-v4/19980101021528-REMSS-L2P_GHRSST-SSTsubs

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/namedarray/core.py:261: UserWarning: Duplicate dimension names present: dimensions {'Grid_Pressure'} appear more than once in dims=('Grid_Targets', 'Grid_Pressure', 'Grid_Pressure'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recommend you rename the dims immediately to become distinct, as most xarray functionality is likely to fail silently if you do not. To rename the dimensions you will need to set the ``.dims`` attribute of each variable, ``e.g. var.dims=('x0', 'x1')``.
  self._dims = self._parse_dimensions(dims)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/namedarray/core.py:261: UserWarning: Duplicate dimension names present: dimensions {'Grid_Pressure'} appear more than once in dims=('Grid_Targets', 'Grid_Pressure', 'Grid_Pressure'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recomme


🔍 [1566] Concept ID: C2599212091-POCLOUD
   🚀 [1566] Starting search for C2599212091-POCLOUD...
   🔗 [1566] Link chosen: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/TOPEX_POSEIDON_GDR_F/c3/TP_GPN_2PfP003_002_19921013_003126_19921013_012737.nc
   📊 [1566] Variables (183): ['time_20hz', 'time_tai', 'l2_record_counter', 'altitude', 'altitude_20hz', 'altitude_cnes', 'altitude_rate_mean_sea_surface', 'delta_ellipsoid_tp_wgs84', 'alt_state_flag_oper', 'alt_state_flag_c_band', 'alt_state_flag_ku_band_status', 'alt_state_flag_c_band_status', 'alt_echo_type', 'num_iterations_ku', 'num_iterations_ku_mle3', 'num_iterations_c', 'mqe_20hz_ku', 'mqe_20hz_ku_mle3', 'mqe_20hz_c', 'range_ku', 'range_20hz_ku', 'range_ku_mle3', 'range_20hz_ku_mle3', 'range_c', 'range_20hz_c', 'range_used_20hz_ku', 'range_used_20hz_ku_mle3', 'range_used_20hz_c', 'range_rms_ku', 'range_rms_ku_mle3', 'range_rms_c', 'range_numval_ku', 'range_numval_ku_mle3', 'range_numval_c', 'range_ku_qual', 'ran

In [14]:
df_valid_vars = df.dropna(subset=["variables"])
df_valid_vars

,concept_id,short_name,entry_title,provider_id,begin_time,end_time,west,south,east,north,links,variables,status,error,scheme
0,C2105092163-LAADS,VNP03IMG,VIIRS/NPP Imagery Resolution Terrain Corrected...,LAADS,2012-01-19T00:00:00.000Z,NaN,-180.000000,-90.000000,180.000000,90.00000,https://data.laadsdaac.earthdatacloud.nasa.gov...,[],ok,None,https
1,C2545314550-LPCLOUD,VNP21,VIIRS/NPP Land Surface Temperature and Emissiv...,LPCLOUD,2012-01-17T00:00:00.000Z,NaN,-180.000000,-90.000000,180.000000,90.00000,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,[],ok,None,https
2,C2105091501-LAADS,VNP02IMG,VIIRS/NPP Imagery Resolution 6-Min L1B Swath 3...,LAADS,2012-01-19T00:00:00.000Z,NaN,-180.000000,-90.000000,180.000000,90.00000,https://data.laadsdaac.earthdatacloud.nasa.gov...,[],ok,None,https
3,C1593392869-LAADS,CLDMSK_L2_MODIS_Aqua,MODIS/Aqua Cloud Mask 5-Min Swath 1000 m,LAADS,2002-07-04T00:00:00.000Z,NaN,-180.000000,-90.000000,180.000000,90.00000,https://data.laadsdaac.earthdatacloud.nasa.gov...,[],ok,None,https
4,C1562021084-LAADS,CLDMSK_L2_VIIRS_SNPP,VIIRS/Suomi-NPP Cloud Mask 6-Min Swath 750 m,LAADS,2012-03-01T00:00:00.000Z,NaN,-180.000000,-90.000000,180.000000,90.00000,https://data.laadsdaac.earthdatacloud.nasa.gov...,[],ok,None,https
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,C2382050573-GHRC_DAAC,ualbmrr2impacts,UAlbany Micro Rain Radar 2 (MRR-2) IMPACTS,GHRC_DAAC,2020-01-30T00:01:00.000Z,2023-02-28T23:59:00.000Z,-73.832439,42.680377,-73.813907,42.68628,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...,[],open_failed,b'CDF\x01\x00\x00\x00\x00' is not the signatur...,https
1930,C2102858144-GHRC_DAAC,ualbparsimpacts,UAlbany Parsivel IMPACTS,GHRC_DAAC,2020-01-30T00:00:00.000Z,2023-02-28T23:58:40.000Z,-73.841900,42.670916,-73.804445,42.69570,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...,"[Prcp_Intensity, Prcp_Start, Wx_Code_Synop, Wx...",ok,None,https
1931,C1995869868-GHRC_DAAC,uiucsndimpacts,Mobile UIUC Soundings IMPACTS V1,GHRC_DAAC,2020-01-18T16:00:00.000Z,2023-02-28T15:06:50.000Z,-88.253000,38.958000,-70.661000,44.70700,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...,"[TC, RH, Time, HAGL, WINDSPD, PRESS, WINDDRN]",ok,None,https
1932,C3247862082-GHRC_DAAC,wbandimpacts,ACHIEVE W-Band Cloud Radar IMPACTS,GHRC_DAAC,2023-01-23T01:00:35.000Z,2023-03-01T00:42:40.000Z,-72.861000,41.368000,-71.655000,42.26800,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...,[],open_failed,b'CDF\x01\x00\x00\x00\x00' is not the signatur...,https


In [16]:
# Save result
df.to_csv("output/cmr_collections_netcdf4_updated_saved_all.csv", index=False)
print(f"\n✅ Updated CSV saved with {df['links'].notna().sum()} links populated.")


✅ Updated CSV saved with 1912 links populated.


In [ ]:
## For grouped hdf-5 files, it does not use datatree (reason is current mechanics of Titiler-CMR).
url = "https://data.laadsdaac.earthdatacloud.nasa.gov/prod-lads/VNP03IMG/VNP03IMG.A2012019.0000.002.2020318135750.nc"
fs = earthaccess.get_fsspec_https_session()
ds = xr.open_datatree(fs.open(url), engine="h5netcdf", decode_times=False)
ds